<a href="https://colab.research.google.com/github/NIKKISAHA/machineLearning/blob/main/chat_bot_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# 1. Prepare the Dataset
questions = ["hello", "how are you", "what is your name", "tell me a joke", "goodbye"]
answers = ["hi there", "i am doing well", "i am a chatbot", "why don't scientists trust atoms? because they make up everything!", "see you later"]

# 2. Tokenize the Data
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(questions + answers)
vocab_size = len(tokenizer.word_index) + 1

encoder_input_data = tokenizer.texts_to_sequences(questions)
decoder_input_data = tokenizer.texts_to_sequences(["\t" + ans for ans in answers])
decoder_target_data = tokenizer.texts_to_sequences([ans + "\n" for ans in answers])

max_encoder_seq_length = max(len(seq) for seq in encoder_input_data)
max_decoder_seq_length = max(len(seq) for seq in decoder_input_data)

encoder_input_data = keras.preprocessing.sequence.pad_sequences(encoder_input_data, maxlen=max_encoder_seq_length, padding='post')
decoder_input_data = keras.preprocessing.sequence.pad_sequences(decoder_input_data, maxlen=max_decoder_seq_length, padding='post')
decoder_target_data = keras.preprocessing.sequence.pad_sequences(decoder_target_data, maxlen=max_decoder_seq_length, padding='post')

decoder_target_one_hot = np.zeros((len(decoder_target_data), max_decoder_seq_length, vocab_size), dtype='float32')
for i, seq in enumerate(decoder_target_data):
    for j, char_idx in enumerate(seq):
        if char_idx > 0:
            decoder_target_one_hot[i, j, char_idx] = 1.

latent_dim = 256
encoder_inputs = keras.layers.Input(shape=(None,))
encoder_embedding = keras.layers.Embedding(vocab_size, latent_dim)(encoder_inputs)
encoder_lstm = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = keras.layers.Input(shape=(None,))
decoder_embedding = keras.layers.Embedding(vocab_size, latent_dim)(decoder_inputs)
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

decoder_dense = keras.layers.Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
# model.compile(optimizer='adam', loss='categorical_crossentropy')
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_one_hot, epochs=200, batch_size=2)

def evaluate_model(model, encoder_input_data, decoder_input_data, decoder_target_one_hot):
    loss = model.evaluate([encoder_input_data, decoder_input_data], decoder_target_one_hot, verbose=0)
    print(f'Model Loss: {loss}')

evaluate_model(model, encoder_input_data, decoder_input_data, decoder_target_one_hot)

# Make Predictions

def make_prediction(model, encoder_input_data, decoder_input_data):
    prediction = model.predict([encoder_input_data, decoder_input_data])
    return prediction

prediction = make_prediction(model, encoder_input_data, decoder_input_data)

# Decode the Prediction

def decode_prediction(prediction, tokenizer):
    decoded_prediction = []
    for seq in prediction:
        decoded_seq = ''
        for probs in seq:
            char_idx = np.argmax(probs)
            if char_idx > 0:
                char = tokenizer.index_word.get(char_idx, '')
                decoded_seq += char
        decoded_prediction.append(decoded_seq)
    return decoded_prediction

decoded_prediction = decode_prediction(prediction, tokenizer)

# Print the Decoded Prediction

for i, question in enumerate(questions):
    print(f'Question: {question}')
    print(f'Answer: {answers[i]}')
    print(f'Prediction: {decoded_prediction[i]}')
    print('---')

Epoch 1/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 206ms/step - accuracy: 0.0305 - loss: 1.0955
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 185ms/step - accuracy: 0.1004 - loss: 1.0679
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 230ms/step - accuracy: 0.0968 - loss: 1.3491
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 198ms/step - accuracy: 0.0335 - loss: 0.8423
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 271ms/step - accuracy: 0.0428 - loss: 1.3658
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 341ms/step - accuracy: 0.0642 - loss: 1.2682
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 259ms/step - accuracy: 0.0553 - loss: 0.9457
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 203ms/step - accuracy: 0.0534 - loss: 0.8976
Epoch 9/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 191ms/step - accuracy: 0.0706 - loss: 0.9255
Epoch 10/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 297ms/step - accuracy: 0.0691 - loss: 1.2200
Epoch 11/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 210ms/step - accuracy: 0.0600 - loss: 1.1611
Epoch 12/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 201ms/step - accuracy:

In [ ]:
def evaluate_model(model, encoder_input_data, decoder_input_data, decoder_target_one_hot):
    loss, accuracy = model.evaluate([encoder_input_data, decoder_input_data], decoder_target_one_hot, verbose=0)
    print(f'Model Loss: {loss}')
    print(f'Model Accuracy: {accuracy}')

evaluate_model(model, encoder_input_data, decoder_input_data, decoder_target_one_hot)


Model Loss: 0.021976029500365257
Model Accuracy: 0.35151517391204834
